In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
from yaml import load as yaml_load
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
spark = SparkSession.builder.master("local").appName("Data cleaning").getOrCreate()

In [4]:
import os
import sys
sys.path.append('../')

In [5]:
sc = spark.sparkContext

In [6]:
import json

In [7]:
from src.data.make_dataset import LoadDataframe

In [8]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [9]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2018')

CPU times: user 352 µs, sys: 66 µs, total: 418 µs
Wall time: 1.14 ms


In [10]:
df_socio = obj_df_loaded.df_socio()

In [11]:
df_crime = obj_df_loaded.df_crime()

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [55]:
df_crime_socio = df_crime.join(df_socio, ['community_area_number'], "inner")

In [56]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType
import re

In [57]:
from astral import Astral
city_name = 'Chicago'
a = Astral()
a.solar_depression = 'civil'
city = a[city_name]
#sun = city.sun(date=date, local=True)

In [58]:
extract_blok = udf(lambda x : re.findall(r"(\w+)$", x)[0], StringType())
isStreet = udf(lambda x :  1 if x in ['ST', 'St', 'st'] else 0)
isAV = udf(lambda x : 1 if x in ['Ave', 'AV'] else 0)
isBLVD = udf(lambda x : 1 if x in ['BLVD'] else 0)
isRD = udf(lambda x : 1 if x in ['RD'] else 0)
isPL = udf(lambda x : 1 if x in ['PL', 'pl'] else 0)
isBROADWAY = udf(lambda x : 1 if x in ['BROADWAY', 'Broadway'] else 0)
isPKWY = udf(lambda x : 1 if x in ['PKWY', 'Pkwy'] else 0)
duration_day = udf(lambda x :  (city.sun(date=x, local=True)['sunset'] - city.sun(date=x, local=True)['sunrise']).total_seconds())

In [42]:
df_crime_socio = df_crime_socio.withColumn("block_extract", extract_blok(df_crime_socio.block))

In [43]:
df_crime_socio = df_crime_socio.withColumn("isStreet", isStreet(df_crime_socio.block_extract)).withColumn("isAV", isAV(df_crime_socio.block_extract)).withColumn("isBLVD", isBLVD(df_crime_socio.block_extract)).withColumn("isRD", isRD(df_crime_socio.block_extract)).withColumn("isPL", isPL(df_crime_socio.block_extract)).withColumn("isBROADWAY", isBROADWAY(df_crime_socio.block_extract)).withColumn("isPKWY", isPKWY(df_crime_socio.block_extract))

In [45]:
df_crime_socio.limit(10).toPandas()

,community_area_number,id,cas_number,date,block,iucr,primary_type,description,location_description,arrest,...,per_capita_income,hardship_index,block_extract,isStreet,isAV,isBLVD,isRD,isPL,isBROADWAY,isPKWY
0,25,10000092,HY189866,2015-03-18 19:44:00,047XX W OHIO ST,041A,ASSAULT_BATTERY,AGGRAVATED: HANDGUN,STREET,false,...,15957,73,ST,1,0,0,0,0,0,0
1,67,10000094,HY190059,2015-03-18 23:00:00,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,...,11317,89,AVE,0,0,0,0,0,0,0
2,39,10000095,HY190052,2015-03-18 22:45:00,044XX S LAKE PARK AVE,0486,ASSAULT_BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,...,35911,26,AVE,0,0,0,0,0,0,0
3,40,10000096,HY190054,2015-03-18 22:30:00,051XX S MICHIGAN AVE,0460,ASSAULT_BATTERY,SIMPLE,APARTMENT,false,...,13785,88,AVE,0,0,0,0,0,0,0
4,25,10000097,HY189976,2015-03-18 21:00:00,047XX W ADAMS ST,031A,THEFT_ROBBERY_BURGLARY,ARMED: HANDGUN,SIDEWALK,false,...,15957,73,ST,1,0,0,0,0,0,0
5,39,10000098,HY190032,2015-03-18 22:00:00,049XX S DREXEL BLVD,0460,ASSAULT_BATTERY,SIMPLE,APARTMENT,false,...,35911,26,BLVD,0,0,1,0,0,0,0
6,68,10000099,HY190047,2015-03-18 23:00:00,070XX S MORGAN ST,0486,ASSAULT_BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,...,11888,94,ST,1,0,0,0,0,0,0
7,38,10000100,HY189988,2015-03-18 21:35:00,042XX S PRAIRIE AVE,0486,ASSAULT_BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,...,23472,57,AVE,0,0,0,0,0,0,0
8,59,10000101,HY190020,2015-03-18 22:09:00,036XX S WOLCOTT AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,true,...,16954,61,AVE,0,0,0,0,0,0,0
9,49,10000104,HY189964,2015-03-18 21:25:00,097XX S PRAIRIE AVE,0460,ASSAULT_BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,false,...,17949,52,AVE,0,0,0,0,0,0,0


In [62]:
df_crime_socio = df_crime_socio.withColumn('duree_day', duration_day(df_crime_socio.date))

In [64]:
df_crime_socio.columns

['community_area_number',
 'id',
 'cas_number',
 'date',
 'block',
 'iucr',
 'primary_type',
 'description',
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'fbi_code',
 'x_coordinate',
 'y_coordinate',
 'year',
 'updated_on',
 'latitude',
 'longitude',
 'location',
 'community_area_name',
 'pct_housing_crowded',
 'pct_households_below_poverty',
 'pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index',
 'duree_day']

In [65]:
df_crime_socio.show(5)

Py4JJavaError: An error occurred while calling o528.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 361, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 236, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 163, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", lin